## Welcome to Bloonet notebook.


#### Run this cell to connect to your GIS and get started:

In [ ]:
from arcgis.gis import GIS
gis = GIS("home")

#### Necessary imports

In [ ]:
import requests
import uuid
import json
import math
import time
import rasterio
import numpy as np
from copy import deepcopy
from zipfile import ZipFile
from arcgis.raster.analytics import copy_raster
from arcgis.map import Map
from typing import Union
from ipywidgets import widgets
from IPython.display import display

#### Constants
Constants containing Bloonet server connection information and the name of the directory where results will be saved.

In [ ]:
SESSION_UUID = uuid.uuid4()
SIMULATION_UUID = uuid.uuid4()
RECEPTION_HEIGHTS = [2]
RESOLUTION = 20
RADIUS = 5000
FREQUENCY = 3600
BWS_API_URL = "https://api.bloonetws.siradel.com"
BWS_DL_URL = "https://dl.bloonetws.siradel.com"

#### Tokens

In [ ]:
ACCESS_TOKEN = None
REFRESH_TOKEN = None
AUTHENTICATION = {
    "url": "https://keycloak.bloonetws.siradel.com/realms/volcanoweb/protocol/openid-connect/token",
    "clientId": "volcano-web-cli",
    "clientSecret": "CLIENT_SECRET",
    "username": "USERNAME",
    "password": "PASSWORD"
}
FOLDER_NAME = "Notebook Bloonet"

#### Required functions
Server call functions with authentication management.

In [ ]:
def call_request(method: str, url: str, authentication_data: dict,
                 files: list = None, json_content: Union[dict, list] = None,
                 params: dict = None, retry: int = 0, timeout=None) -> requests.Response:
    """
    @summary: call requests with args for use authenticate
    @param method: {str} method request
    @param url: {str} url request
    @param authentication_data: {dict} authentication data
    @param files: {list} files request
    @param json_content: {dict} json request
    @param params: {dict} params request
    @param retry: {int} number of retry
    @param timeout: {Any} timeout request
    @return: {request.Response} response of request
    """

    headers = None

    if authentication_data is None:
        print("Authentication data needed")
    token = get_access_token(authentication_data)
    headers = {"Authorization": f"Bearer {token}"}

    res = None
    if method.upper() == "POST":
        res = requests.post(url=url, files=files, json=json_content, params=params,
                            timeout=timeout, headers=headers)
    elif method.upper() == "PUT":
        res = requests.put(url=url, files=files, json=json_content, params=params,
                           timeout=timeout, headers=headers)
    elif method.upper() == "GET":
        res = requests.get(url=url, files=files, json=json_content, params=params,
                           timeout=timeout, headers=headers)
    elif method.upper() == "DELETE":
        res = requests.delete(url=url, files=files, json=json_content, params=params,
                              timeout=timeout, headers=headers)
    else:
        raise ValueError("Method name %S not implemented", method.upper())

    # If access_token expires, refresh the token one time and recall requests
    if res.status_code == 403 and retry == 0:
        refresh_token(authentication_data)
        res = call_request(
            method,
            url,
            authentication_data,
            files,
            json_content,
            params,
            1,
            timeout
        )

    return res


def get_access_token(authentication_data: dict) -> str:
    """
    @summary: Retrieve an access token
    @param authentication_data: {dict} authentication data
    @return: {str} access token provided
    """

    global ACCESS_TOKEN, REFRESH_TOKEN
    if ACCESS_TOKEN is not None:
        return ACCESS_TOKEN
    payload = {
        "grant_type": "password",
        "client_id": authentication_data["clientId"],
        "client_secret": authentication_data["clientSecret"],
        "username": authentication_data["username"],
        "password": authentication_data["password"]
    }

    response = requests.post(
        authentication_data["url"],
        data=payload
    )

    json_response = response.json()

    if response.status_code == 200:
        ACCESS_TOKEN = json_response["access_token"]
        REFRESH_TOKEN = json_response["refresh_token"]
    else:
        raise RuntimeError("Error call get access token: %s", json_response["error_description"])
        sys.exit(errno.EINVAL)
    return ACCESS_TOKEN


def refresh_token(authentication_data: dict) -> None:
    """
    @summary: Refresh the access token
    @param authentication_data: {dict} authentication data
    """

    global ACCESS_TOKEN, REFRESH_TOKEN
    if REFRESH_TOKEN is None:
        raise RuntimeError("Error call refresh token : refresh token is empty")

    payload = {
        "grant_type": "refresh_token",
        "client_id": authentication_data["clientId"],
        "client_secret": authentication_data["clientSecret"],
        "refresh_token": REFRESH_TOKEN
    }

    response = requests.post(
        authentication_data["url"],
        data=payload
    )

    json_response = response.json()

    if response.status_code == 200:
        ACCESS_TOKEN = json_response["access_token"]
        REFRESH_TOKEN = json_response["refresh_token"]
    else:
        raise RuntimeError("Error call get refresh token: %s", json_response["error_description"])

#### User input
Indicate the name of the ArcGIS map on which the Feature Layer network is positioned.

In [ ]:
valid_input = False
while valid_input != True:
    map_name = input("Which map would you like to work on ? ")
    map_obj = gis.content.search(
        query = f"title:{map_name}",
        item_type = "Web Map",
        max_items=1
    )
    if len(map_obj) > 0:
        bws_map = map_obj[0]
        bws_webmap = Map(bws_map)
        valid_input = True
    else:
        print("Unknown Web Map")

#### Propagation model selection
Select one of the public propagation model used for calculation in the Bloonet API.

In [ ]:
# Retrieve public propagation models
public_models_list = call_request(
    method="GET",
    url=f"{BWS_API_URL}/propagationmodels",
    authentication_data=AUTHENTICATION
).json()
PUBLIC_MODELS_NAME = ["fixed wireless access", "mobility"]
public_models = [model for model in public_models_list if model["name"].lower() in PUBLIC_MODELS_NAME and model["type"] is None]

while True:
    print("Please select a propagation model:")
    for i, public_model in enumerate(public_models, start=1):
        print(f"{i}. {public_model['name']}")

    try:
        choice = int(input("Enter the number of your choice : "))
        if 1 <= choice <= len(public_models):
            print(f"You have chosen: {public_models[choice - 1]['name']}")
            model_uuid = public_models[choice - 1]["uuid"]
            break
        else:
            print("Invalid choice")
    except ValueError:
        print("Please enter a number")


#### Antenna selection
Select one of the public antenna used for calculation in the Bloonet API.

In [ ]:
# Retrieve public antennas
public_antennas_list = call_request(
    method="GET",
    url=f"{BWS_API_URL}/antennas",
    authentication_data=AUTHENTICATION
).json()
PUBLIC_ANTENNAS_NAME = [
    "tarana-bn-3ghz-compact-r0",
    "tarana-bn-5ghz-r1",
    "tarana-bn-6ghz-r2",
    "tarana-bn-6ghz-x2-r2"
]
public_antennas = [antenna for antenna in public_antennas_list if antenna["name"].lower() in PUBLIC_ANTENNAS_NAME]

while True:
    print("Please select an antenna:")
    for i, public_antenna in enumerate(public_antennas, start=1):
        print(f"{i}. {public_antenna['name']}")

    try:
        choice = int(input("Enter the number of your choice : "))
        if 1 <= choice <= len(public_antennas):
            print(f"You have chosen: {public_antennas[choice - 1]['name']}")
            antenna_uuid = public_antennas[choice - 1]["uuid"]
            break
        else:
            print("Invalid choice")
    except ValueError:
        print("Please enter a number")

#### Session creation
Creating a session in the Bloonet API.

In [ ]:
# Session information
session = {
    "uuid": str(SESSION_UUID),
    "name": "session_sample_3DTWorld_esri",
    "description": "Session sample 3DTWorld from ESRI Notebook"
}

# Post session to BWS server
result = call_request(
    method="POST",
    url=f"{BWS_API_URL}/sessions",
    authentication_data=AUTHENTICATION,
    json_content=session
).json()

print(result)

#### Network list creation
Creation of the network list used for the calculation from the Feature Layer data on the map.

In [ ]:
network_list = []

# Retrieve Feature layer from input web map
feature_layers = list(filter(lambda x: (x.properties.type == "Feature Layer"), bws_webmap.content.layers))
feature_layer_item = gis.content.get(feature_layers[0].properties.serviceItemId)
feature_layer = feature_layer_item.layers[0]

query_result = feature_layer.query(out_sr='4326')

# Update Feature layer easting and northing by geometry
features_for_update = []
for feature in query_result.features:
    feature_to_be_updated = deepcopy(feature)
    feature_to_be_updated.attributes["long"] = float(feature_to_be_updated.geometry["x"])
    feature_to_be_updated.attributes["lat"] = float(feature_to_be_updated.geometry["y"])
    features_for_update.append(feature_to_be_updated)

feature_layer.edit_features(updates=features_for_update)

# Retrieve network informations in correct epsg
query_result2 = feature_layer.query(out_sr='4326')

for feat in query_result2.features:
    feat.attributes["long"] = float(feat.geometry["x"])
    feat.attributes["lat"] = float(feat.geometry["y"])
    # Replace _ by space in feature attributes keys
    feature_attributes = { x.replace('_', ' ') : y
                     for x, y in feat.attributes.items()}
    if "ObjectId" in feature_attributes:
        del feature_attributes["ObjectId"]
    network_list.append(feature_attributes)

print(network_list)

#### Simulation creation
Creating a simulation for Bloonet API calculation.

In [ ]:
# Create simulation object
propagation_request = {}
propagation_list = []
for network in network_list:
    # Bbox calculation
    r_earth = 6378.137  # radius of the earth in kilometer
    m = (1 / ((2 * math.pi / 360) * r_earth)) / 1000  # 1 meter in degree
    lat_min = float(network["lat"]) - (RADIUS * m)
    lat_max = float(network["lat"]) + (RADIUS * m)
    long_min = float(network["long"]) - (RADIUS * m) / math.cos(lat_min * (math.pi / 180))
    long_max = float(network["long"]) + (RADIUS * m) / math.cos(lat_max * (math.pi / 180))

    propagation_list.append({
        "baseStation": {
            "x": float(network["long"]),
            "y": float(network["lat"]),
            "z": float(network["height"]),
            "epsgCode": 4326,
            "zmeaning": "ZMEANING_GROUND",
            "azimuth": float(network["azimuth"]) if "azimuth" in network else 0,
            "downtilt": float(network["downtilt"]) if "downtilt" in network else 0,
            "carrierFrequency": FREQUENCY,
            "description": "",
            "sessionUuid": str(SESSION_UUID),
            "name": network["name"],
            "networkId": network["id"],
            "transmitPower": float(network["power"]),
            "antennaUuid": antenna_uuid
        },
        "userEquipments": [{
            "sessionUuid": str(SESSION_UUID),
            "description": "",
            "zmeaning": "ZMEANING_GROUND",
            "type": "AREA",
            "name": network["name"],
            "heights": RECEPTION_HEIGHTS,
            "coordinates": {
                "xmin": min(long_min, long_max),
                "xmax": max(long_min, long_max),
                "ymin": min(lat_min, lat_max),
                "ymax": max(lat_min, lat_max),
                "resolution": RESOLUTION,
                "epsgCode": 4326
            }
        }],
        "propagationModelUuid": model_uuid
    })

    
simulation_request = {
    "uuid": str(SIMULATION_UUID),
    "name": "session_sample_3DTWorld_esri_simulation",
    "calculationSessionUuid": str(SESSION_UUID),
    "propagationRequest": {
        "propagationScenarios": propagation_list,
        "resultTypes": ["RECEIVED_POWER"],
        "heights": RECEPTION_HEIGHTS,
        "zmeaning": "ZMEANING_GROUND"
    },
    "postprocessingRequest": {
        "resolution": RESOLUTION,
        "computationType": "Custom",
        "resultTypes": [
            "BEST_SERVER",
            "DL_BEST_SIGNAL"
        ]
    }
}

multipart_form_data = [("json", (None, json.dumps(simulation_request), "application/json")), *[]]

# Post simulation to BWS server
simulation = call_request(
    method="POST",
    url=f"{BWS_API_URL}/simulations",
    authentication_data=AUTHENTICATION,
    files=multipart_form_data
).json()

print(simulation)

#### Pull simulation status
Retrieve simulation calculation progress from Bloonet API until calculation is complete.

In [ ]:
# Each 5 sec, retrieve the status of the simulation and stop when finished
while True:
    time.sleep(5)
    response = call_request(
        method="GET",
        url=f"{BWS_API_URL}/simulations/{str(SIMULATION_UUID)}/status",
        authentication_data=AUTHENTICATION
    ).json()
    
    print(response["progress"])
    
    if response["state"] == "DONE":
        print(f"Simulation {str(SIMULATION_UUID)} finished with state {response['state']}")
        break
    if response["state"] == "ERROR":
        errorMsg = f"Simulation {str(SIMULATION_UUID)} finished with state {response['state']} with message {response['error']} : {response['errorMessages']}"
        raise RuntimeError(errorMsg)

#### Download simulation results
Downloading simulation results from the Bloonet API.

In [ ]:
# Retrieve the simulation results
res = call_request(
    method="GET",
    url=f"{BWS_API_URL}/simulations/{str(SIMULATION_UUID)}/results",
    authentication_data=AUTHENTICATION
)

results = res.json()

if res.status_code in (404, 400):
    raise RuntimeError(f"Error simulation {SIMULATION_UUID}: {results['message']}")

rasters = []
for result in results:
    # Download the tif files
    current_tiff = call_request(
        method="GET",
        url=f"{BWS_DL_URL}/results/{str(result['uuid'])}/download",
        authentication_data=AUTHENTICATION
    )
    
    if current_tiff.status_code != 200:
        raise RuntimeError(f"Error downloading {result['fileName']}")
    
    # Save this tif files in tmp folder
    with open(f"/tmp/{result['fileName']}", "wb") as result_file:
        result_file.write(current_tiff.content)
    
    filename = result["fileName"].rsplit(".", 1)[0]
    
    rasters.append({
        "name": filename.replace("-", "_").replace(".", "_"),
        "path": f"/tmp/{result['fileName']}"
    })

print(rasters)

#### Map update
ArcGIS map update :
- Removal of old best signal and best server layers from the map and ArcGIS content
- Creation of layers from new rasters previously downloaded
- Add these layers to the map
- Update layer properties (coloring, popup elements)
- Display map in Notebook

In [ ]:
init_time = time.time()
    
# Keep only feature layers
layers_to_keep = list(layer for layer in bws_webmap.content.layers if "Feature Layer" in layer.properties.type)

bws_webmap.content.remove_all()

for raster in rasters:
    # Remove old rasters
    old_raster_obj = gis.content.search(
        query = f"title:{raster['name']}_layer",
        item_type = "Imagery Layer",
        max_items=1
    )
    if old_raster_obj:
        print(f"Remove {old_raster_obj} from content")
        gis.content.delete_items(old_raster_obj, permanent=True)

    out_raster = raster['path']
    
    # NoData management for best signal raster
    if "best_signal" in raster['name']:
        out_raster = f"/tmp/{raster['name']}_op.tif"
        no_data_value = -999

        with rasterio.open(raster["path"], "r+") as src:
            src.nodata = no_data_value
            with rasterio.open(out_raster, 'w',  **src.profile) as dst:
                for i in range(1, src.count + 1):
                    band = src.read(i)
                    band = np.where(band<-999,no_data_value,band)
                    dst.write(band,i)
    
    # Create new raster
    raster_layer = copy_raster(input_raster=out_raster,
                               output_name=f"{raster['name']}_layer",
                               folder=FOLDER_NAME,
                               gis=gis)
    raster_layer_item = gis.content.get(raster_layer.itemid)
    bws_webmap.content.add(raster_layer_item)

create_rasters_time = time.time()
print(f"Create rasters : {create_rasters_time - init_time}")

for layer in layers_to_keep:
    bws_webmap.content.add(layer)

bws_webmap.update()

wm_item = gis.content.get(bws_map.itemid)
wm_data = wm_item.get_data()
layers = wm_data['operationalLayers']

# Applies properties to map layers according to type (best signal, best server)
# These properties allow to configure the color palette for displaying the layer
# as well as the activation of the selection popup to display pixel values.
for layer in layers:
    if layer["layerType"] == "ArcGISFeatureLayer":
        layer_properties = {
            'disablePopup':False
        }
    else:
        layer_properties = {
            'disablePopup':False,
            'maxScale':0,
            'minScale':0,
            'layerDefinition': {
                'drawingInfo': {
                    'renderer': {
                        'colorRamp': {
                            'type':'multipart',
                            'colorRamps':[
                                {
                                    'type':'algorithmic',
                                    'algorithm':'esriHSVAlgorithm',
                                    'fromColor':[11,44,122,255],
                                    'toColor':[32,153,143,255]
                                },
                                {
                                    'type':'algorithmic',
                                    'algorithm':'esriHSVAlgorithm',
                                    'fromColor':[32,153,143,255],
                                    'toColor':[0,219,0,255]
                                },
                                {
                                    'type':'algorithmic',
                                    'algorithm':'esriHSVAlgorithm',
                                    'fromColor':[0,219,0,255],
                                    'toColor':[255,255,0,255]
                                },
                                {
                                    'type':'algorithmic',
                                    'algorithm':'esriHSVAlgorithm',
                                    'fromColor':[255,255,0,255],
                                    'toColor':[237,161,19,255]
                                },
                                {
                                    'type':'algorithmic',
                                    'algorithm':'esriHSVAlgorithm',
                                    'fromColor':[237,161,19,255],
                                    'toColor':[194,82,60,255]
                                }
                            ]
                        },
                        'computeGamma':False,
                        'dra':True,
                        'gamma':[1],
                        'numberOfStandardDeviations':2,
                        'max':255,
                        'min':0,
                        'useGamma':False,
                        'stretchType':'minMax',
                        'type':'rasterStretch'
                    }
                }
            },
            'popupInfo': {
                'popupElements': [
                    {
                        'type': 'fields',
                        'title': 'Bloonet best signal coverage' if ('best_signal' in layer['title']) else 'Bloonet best server coverage',
                        'description': 'Pixel value corresponds to the dB value of the strongest signal' if ('best_signal' in layer['title']) else 'Pixel value corresponds to the cell ID with the strongest signal',
                        'fieldInfos':[
                            {
                                'fieldName':'Raster.ServicePixelValue',
                                'format': {
                                    'digitSeparator':True,
                                    'places':0
                                },
                                'isEditable':False,
                                'label':'Pixel Value',
                                'visible':True
                            }
                        ]
                    }
                ]
            }
        }
    layer.update(layer_properties)
wm_properties = {'text': json.dumps(wm_data)}
wm_item.update(item_properties=wm_properties)

map_updated = Map(wm_item)

map_updated